<a href="https://colab.research.google.com/github/Rustydinoco/ML_Learning/blob/master/Streamlit_LSTM_S%26P_500_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [8]:
import joblib
from sklearn.preprocessing import MinMaxScaler

joblib.dump(MinMaxScaler, 'MinMaxScaler.joblib')


['MinMaxScaler.joblib']

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
import streamlit as st
from pyngrok import ngrok
import getpass

class LSTMMOdel(nn.Module):
  def __init__(self, input_size = 1, hidden_layer_size  = 100, output= 1):
    super().__init__()
    self.hidden_layer_size = hidden_layer_size
    self.lstm = nn.LSTM(input_size, hidden_layer_size)
    self.linear = nn.Linear(hidden_layer_size, output)

  def forward(self, input_seq):
    lstm_out, _ = self.lstm(input_seq)
    prediction = self.linear(lstm_out[:, -1, :])
    return prediction


@st.cache_resource
def load_data(ticker):
  df = yf.download(ticker, start="2010-01-01")
  return df

@st.cache_resource
def load_model_and_scaler():
  try:
      model = LSTMModel()
      model.load_state_dict()(torch.load('model_LSTM_S&P500.pth'))
      model.eval()

      scaler = joblib.load('MinMaxScaler.joblib')
      return model, scaler
  except Exception as e:
      st.error("File Not Found")
      return None, None


st.title("S&P 500 Stock Prediction")

data = load_data("SPY")
model, scaler = load_model_and_scaler()

if model is not None:
  st.subheader("Close Price S&P 500")
  fig = go.Figure()
  fig.add_trace(go.scatter(x= data["Date"], y= data["Close"], name= "S&P 500"))
  fig.layout.update(title_text = "S&P 500 Price Movement", xaxis_rangeslider_visible = True)
  st.plotly_chart(fig)